In [1]:
!unzip /home/aistudio/data/data78992/lcqmc.zip -d /home/aistudio/data/
!unzip /home/aistudio/data/data78992/paws-x-zh.zip -d /home/aistudio/data/
!unzip /home/aistudio/data/data78992/bq_corpus.zip -d /home/aistudio/data/

Archive:  /home/aistudio/data/data78992/lcqmc.zip


   creating: /home/aistudio/data/lcqmc/
  inflating: /home/aistudio/data/lcqmc/train.tsv  


   creating: /home/aistudio/data/__MACOSX/
   creating: /home/aistudio/data/__MACOSX/lcqmc/
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._train.tsv  
  inflating: /home/aistudio/data/lcqmc/dev.tsv  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._dev.tsv  
  inflating: /home/aistudio/data/lcqmc/License.pdf  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._License.pdf  
  inflating: /home/aistudio/data/lcqmc/test.tsv  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._test.tsv  
  inflating: /home/aistudio/data/lcqmc/User_Agreement.pdf  
  inflating: /home/aistudio/data/__MACOSX/lcqmc/._User_Agreement.pdf  
  inflating: /home/aistudio/data/__MACOSX/._lcqmc  


Archive:  /home/aistudio/data/data78992/paws-x-zh.zip


   creating: /home/aistudio/data/paws-x-zh/
  inflating: /home/aistudio/data/paws-x-zh/train.tsv  


   creating: /home/aistudio/data/__MACOSX/paws-x-zh/
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._train.tsv  
  inflating: /home/aistudio/data/paws-x-zh/dev.tsv  
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._dev.tsv  
  inflating: /home/aistudio/data/paws-x-zh/License.pdf  
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._License.pdf  
  inflating: /home/aistudio/data/paws-x-zh/test.tsv  
  inflating: /home/aistudio/data/__MACOSX/paws-x-zh/._test.tsv  
  inflating: /home/aistudio/data/__MACOSX/._paws-x-zh  


Archive:  /home/aistudio/data/data78992/bq_corpus.zip


   creating: /home/aistudio/data/bq_corpus/
  inflating: /home/aistudio/data/bq_corpus/train.tsv  


   creating: /home/aistudio/data/__MACOSX/bq_corpus/
  inflating: /home/aistudio/data/__MACOSX/bq_corpus/._train.tsv  
  inflating: /home/aistudio/data/bq_corpus/dev.tsv  
  inflating: /home/aistudio/data/__MACOSX/bq_corpus/._dev.tsv  
  inflating: /home/aistudio/data/bq_corpus/License.pdf  
  inflating: /home/aistudio/data/__MACOSX/bq_corpus/._License.pdf  
  inflating: /home/aistudio/data/bq_corpus/test.tsv  
  inflating: /home/aistudio/data/__MACOSX/bq_corpus/._test.tsv  
  inflating: /home/aistudio/data/bq_corpus/User_Agreement.pdf  
  inflating: /home/aistudio/data/__MACOSX/bq_corpus/._User_Agreement.pdf  
  inflating: /home/aistudio/data/__MACOSX/._bq_corpus  


## 1. Load dataset

In [2]:
def load_dataset(fpath, num_row_skip=0):

    def read(fp):
        data = open(fp)

        for _ in range(num_row_skip):
            next(data)

        if "test" in fp:
            for line in data:
                line = line.strip().split('\t')
                yield line[0], line[1]
        else:
            for line in data:
                line = line.strip().split('\t')
                if len(line) == 3:
                    yield line[0], line[1], int(line[2])

    if isinstance(fpath, str):
        return list(read(fpath))
    elif isinstance(fpath, (list, tuple)):
        return [list(read(fp)) for fp in fpath]
    else:
        raise TypeError("Input fpath must be a str or a list/tuple of str")

In [3]:
train_set, dev_set, test_set = load_dataset(['./data/paws-x-zh/train.tsv', './data/paws-x-zh/dev.tsv', './data/paws-x-zh/test.tsv'])
# len(train_set), len(dev_set), len(test_set)
train_set = train_set + dev_set

## 2. Transform text

In [4]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import ErnieGramModel as SeqClfModel
from paddlenlp.transformers import ErnieGramTokenizer as PTMTokenizer
import numpy as np


MODEL_NAME = "ernie-gram-zh"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    label = np.array([label], dtype="int64")
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset,
                      trans_fn,
                      batchify_fn,
                      test=False,
                      batch_size=128,
                      shuffle=True,
                      sampler=BatchSampler):

    if test:
        dataset = [d + (0,) for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)

    dataset.map(trans_fn)
    batch_sampler = sampler(dataset,
                            shuffle=shuffle,
                            batch_size=batch_size)

    dataloder = DataLoader(dataset,
                           batch_sampler=batch_sampler,
                           collate_fn=batchify_fn)

    return dataloder

[2022-03-25 04:00:00,860] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie_gram_zh/vocab.txt and saved to /home/aistudio/.paddlenlp/models/ernie-gram-zh

[2022-03-25 04:00:00,863] [    INFO]

 - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_gram_zh/vocab.txt

  0%|          | 0/78 [00:00<?, ?it/s]

100%|██████████| 78/78 [00:00<00:00, 29926.43it/s]

In [5]:
max_seq_length = 128; batch_size = 64
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
# dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False, test=True, batch_size=batch_size)

## 3. Model building

In [6]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=2):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        logits = self.fc1(embd)
        return logits

In [7]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 10
weight_decay = 0.0
warmup_proportion = 0.0
lr_scheduler = LinearDecayWithWarmup(5e-5, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(),
    learning_rate=lr_scheduler,
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [8]:
ptm = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(ptm)
model = get_model(model)

[2022-03-25 04:00:01,055] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie_gram_zh/ernie_gram_zh.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-gram-zh

[2022-03-25 04:00:01,057] [    INFO]

 - Downloading ernie_gram_zh.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_gram_zh/ernie_gram_zh.pdparams

  0%|          | 0/583566 [00:00<?, ?it/s]

...

 97%|█████████▋| 565567/583566 [00:08<00:00, 69243.70it/s]

 98%|█████████▊| 572509/583566 [00:08<00:00, 69123.73it/s]

 99%|█████████▉| 580083/583566 [00:08<00:00, 70975.90it/s]

100%|██████████| 583566/583566 [00:08<00:00, 69517.83it/s]

W0325 04:00:09.575878   255 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0325 04:00:09.579895   255 device_context.cc:465] device: 0, cuDNN Version: 7.6.


## 4. Model training

In [9]:
model.fit(train_loader, epochs=epoch, verbose=2, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.

Epoch 1/10

step 100/799 - loss: 0.7095 - acc: 0.5420 - 411ms/step


step 200/799 - loss: 0.5724 - acc: 0.6252 - 409ms/step


step 300/799 - loss: 0.4221 - acc: 0.6872 - 409ms/step


...

step 400/799 - loss: 0.0393 - acc: 0.9905 - 412ms/step


step 500/799 - loss: 0.0050 - acc: 0.9901 - 412ms/step


step 600/799 - loss: 0.0103 - acc: 0.9897 - 413ms/step


step 700/799 - loss: 0.0034 - acc: 0.9897 - 412ms/step


step 799/799 - loss: 0.0251 - acc: 0.9895 - 412ms/step


## 5. Prediction

In [10]:
import paddle.nn.functional as F


predictions = []
logits = model.predict(test_loader)

for batch in logits[0]:
    batch = paddle.to_tensor(batch)
    probs = F.softmax(batch, axis=1)
    preds = paddle.argmax(probs, axis=1).numpy().tolist()
    predictions.extend(preds)

Predict begin...

step  2/32 [>.............................]

 - ETA: 7s - 240ms/step



step  4/32 [==>...........................]

...

 - ETA: 0s - 173ms/step



step 32/32 [==============================]

 - 168ms/step          


Predict samples: 2000

In [11]:
with open('paws-x.tsv', 'w') as f:
    f.write("index\tprediction")
    for idx, p in enumerate(predictions):
        f.write(f"\n{idx}\t{p}")
    f.close()